In [1]:
library(arrow)
library(czso)
library(tidyverse)


Some features are not enabled in this build of Arrow. Run `arrow_info()` for more information.


Attaching package: ‘arrow’


The following object is masked from ‘package:utils’:

    timestamp


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ lubridate::duration() masks arrow::duration()
✖ dplyr::filter()       masks stats::filter()
✖ dplyr::lag()          masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
nzu_raw <- read_parquet("../../data/mzp-cr/nzu-consolidated.parquet") |>
  filter(
    # Only one case is missing the amount.
    !is.na(AmountPaid),
    # Drop applications from unspecified municipalities right away.
    !is.na(ApplicantCity),
    # There are about three cases where the categories span multiple lines but are
    # incorrectly parsed by PDF-to-XLSX. Let's just drop them from the analysis.
    !str_ends(Categories, ",")
  ) |>
  mutate(
    # Quick clean up of some obvious mistakes and inconsistencies in municipality names.
    ApplicantCity = str_replace_all(
      ApplicantCity,
      c(
        # Collapse multiple horizontal space into a single space.
        "\\s\\s+" = " ",
        # Large cities with districts.
        "^(Praha|Brno|České Budějovice)\\b.+" = "\\1",
        # Incorrect input or parsing of hyphenated names.
        "(\\w)\\s*- (\\w)" = "\\1-\\2",
        # Postal code in municipality name.
        "\\d{5}\\s*(.+)$" = "\\1",
        # Common abbreviations.
        "\\bn\\." = "nad",
        "pod Sv\\." = "pod Svatým"
      )
    ),
    # Unify region names with CZSO.
    PropertyRegion = case_when(
      PropertyRegion == "Praha" ~ "Hlavní město Praha",
      PropertyRegion == "Vysočina" ~ "Kraj Vysočina",
      .default = paste(PropertyRegion, "kraj")
    )
  )


## Kategorizace opatření


In [8]:
categories_regex <- c(
  "^E1$|^Bonus.*" = "Bonus",
  ".+CZT$|^C(19|29)$" = "CZT",
  "^.+Zalivka(\\+WC)?$|^.+SedaVoda\\+?$" = "Dešťovka",  # NZÚ 2021+ (D3), 2023+ (D2)
  "^D[34]-E-mobilita$" = "Ekomobilita",  # D4: NZÚ 2021+, D3: NZÚ 2023+
  ".+SOL\\+?$|^C3[12]$|^C3R\\d+$" = "Fototermika",
  ".+FV.*|^C3[3-9]$" = "FVE",
  "^C([12][34])$|.+Kamna.*" = "Kamna",
  "^C([12][12])$|.+Kotel.*" = "Kotel",
  "^B\\d+$|^B-(Pasiv|Zaklad).*" = "Novostavba",
  "^D[1-4]$|^E-.+" = "Projektová podpora",  # (a) NZÚ 2013, (b) NZÚ 2023+
  "^D1-(IQ|Manual)$" = "Stínění",  # NZÚ 2021+
  # FIXME: C3-FVE+TČ se asi matchne na FVE.
  ".+T[CČ].*|^C[12][5-8]$" = "TČ",
  "^C7[12]?$" = "Teplo z odpadní vody",
  "^C4.*" = "Větrání",
  ".*vymena otvoru.*" = "Výměna otvoru",
  "^A-(Dilci|Komplex|Optimalni|Pamatky|Zaklad)$|^L-zateplení$|^A\\d+$" = "Zateplení",
  "^C5.*|^C6$|^C8[1-5]$" = "Zdroje energie",
  # NOTE: Přesunuto do předchozího.
  # "^C6$" = "Zdroje energie_zvýhodnění",
  "^D[12]-.*(ploch|sikm)[aá]$" = "Zelená střecha"  # D2: NZÚ 2021+, D1: NZÚ 2023+
)

all_categories <- nzu_raw |>
  select(CallID, Categories) |>
  separate_longer_delim(Categories, delim = ", ") |>
  distinct() |>
  mutate(
    CategoryDetail = str_replace_all(Categories, categories_regex),
    # Derive top-level categorisation from the lower-level taxonomy.
    CategoryMain = case_match(
      CategoryDetail,
      c("Adaptace/mitigace", "Dešťovka", "Ekomobilita", "Stínění", "Zelená střecha") ~ "Adaptace/mitigace",
      c("Výměna otvoru", "Zateplení") ~ "Zateplení",
      c("CZT", "FVE", "Fototermika", "Kamna", "Kotel", "TČ", "Teplo z odpadní vody", "Větrání", "Zdroje energie") ~
        "Zdroje energie",
      .default = CategoryDetail
    )
  )


In [12]:
nzu_categorised <- nzu_raw |>
  separate_longer_delim(Categories, delim = ", ") |>
  left_join(all_categories, join_by(CallID, Categories)) |>
  group_by(CallID, ApplicationID, ApplicantCity, PropertyRegion, AmountPaid, DatePaid) |>
  summarise(
    CategoriesMain = paste(unique(CategoryMain), collapse = "|"),
    CategoriesDetail = paste(unique(CategoryDetail), collapse = "|"),
    # Re-combine original category codes.
    CategoriesOriginal = paste(Categories, collapse = "|"),
    .groups = "drop"
  )

write_parquet(nzu_categorised, "../../data/mzp-cr/nzu-categorised.parquet")


## Deduplikace a kódování obcí


In [17]:
# Table of municipalities, the basic territorial administrative units in Czechia.
municipalities_czso <- czso_get_table("struktura_uzemi_cr") |>
  select(obec_text, obec_kod, orp_text, orp_csu_cis65_kod, kraj_text) |>
  mutate(obec_text_lower = tolower(obec_text))

# Find duplicate municipality names in Czechia.
duplicate_munis <- municipalities_czso |>
  group_by(obec_text) |>
  filter(n() > 1) |>
  ungroup() |>
  distinct(obec_text, obec_text_lower, kraj_text)

# Find municipalities that share the same name WITHIN one region.
duplicate_munis_regional <- municipalities_czso |>
  group_by(kraj_text, obec_text) |>
  filter(n() > 1) |>
  ungroup() |>
  distinct(obec_text, obec_text_lower, kraj_text)


In [18]:
nzu_categorised <- nzu_categorised |>
  # Temporary adjustment for easier matching with the CZSO dataset.
  mutate(ApplicantCityLower = tolower(ApplicantCity))

# Applications from municipalities that match uniquely with CZSO
# (based on municipality and region name combination where
# the combination is unique).
apps_with_matched_unique_muni <- nzu_categorised |>
  # Discard applications from municipalities that are not unique within
  # the property region.
  anti_join(
    duplicate_munis_regional,
    join_by(ApplicantCityLower == obec_text_lower, PropertyRegion == kraj_text)
  ) |>
  # Join with CZSO based on municipality×region name combination.
  inner_join(
    municipalities_czso,
    join_by(
      ApplicantCityLower == obec_text_lower,
      PropertyRegion == kraj_text
    )
  )

# Applications from municipalities with unique names (across Czechia)
# where the region gets reassigned.
apps_with_rematched_region <- nzu_categorised |>
  # Subtract already matched applications.
  anti_join(
    apps_with_matched_unique_muni,
    join_by(CallID, ApplicationID)
  ) |>
  # Join on unique municipality names.
  inner_join(
    anti_join(municipalities_czso, duplicate_munis, join_by(obec_text)),
    join_by(ApplicantCityLower == obec_text_lower)
  )


In [19]:
nzu_czso <- bind_rows(
    apps_with_matched_unique_muni,
    apps_with_rematched_region
  ) |>
  transmute(
    CallID,
    ApplicationID,
    City = obec_text,
    CityCode = obec_kod,
    ORP = orp_text,
    ORPCode = orp_csu_cis65_kod,
    Region = if_else(!is.na(kraj_text), kraj_text, PropertyRegion),
    PropertyRegion = PropertyRegion,
    CategoriesMain,
    CategoriesDetail,
    AmountPaid,
    DatePaid
  ) |>
  arrange(CallID, ApplicationID)

# Make sure that the primary key is preserved, that there are no duplicate applications
stopifnot(nrow(nzu_czso) == nrow(distinct(nzu_czso, CallID, ApplicationID)))

write_parquet(nzu_czso, "../../data/mzp-cr/nzu-czso.parquet")
